## PDF Yüklemesinin gerçekleştirilmesi

In [6]:
# Önce kafa karıştıran her şeyi siliyoruz
%pip uninstall -y langchain langchain-community langchain-core langchain-google-genai langchain-openai

# Şimdi hepsinin birbirine uyumlu EN SON sürümünü kuruyoruz
# "langchain" paketini eklemek zorunludur, yoksa chains modülü gelmez!
%pip install -U langchain langchain-community langchain-core langchain-google-genai

Found existing installation: langchain 1.2.0Note: you may need to restart the kernel to use updated packages.



Uninstalling langchain-1.2.0:
  Successfully uninstalled langchain-1.2.0
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langchain-core 1.2.5
Uninstalling langchain-core-1.2.5:
  Successfully uninstalled langchain-core-1.2.5
Found existing installation: langchain-google-genai 4.1.2
Uninstalling langchain-google-genai-4.1.2:
  Successfully uninstalled langchain-google-genai-4.1.2
  Using cached langchain-1.2.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_google_genai-4.1.2-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-1.2.0-py3-none-any.whl (102 kB)
Using cached langchain_core-1.2.5-py3-none-any.whl (484 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached lan

In [7]:
# Bu satırı ekle ve çalıştır:
from langchain_community.document_loaders import PyPDFLoader
file_path = "2501.00536v3.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [8]:
len(data)

14

## Veriyi Parçalarına Ayırma (Chunking)

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [10]:
print(f"Number of documents after chunking: {len(docs)}")

Number of documents after chunking: 65


## Google GenAI Embeddings'i kullanarak Embedding Oluşturma İşlemi

In [11]:
load_dotenv()

True

In [46]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello world!")
vector[:5]


[0.013577501, -0.018360399, -0.054512024, -0.009611334, -0.018534755]

In [47]:
## ChromaDB Üzerine Kayıt İşlemi

In [48]:
from langchain_chroma import Chroma

In [49]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [51]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k":10})

In [52]:
retriever_docs = retriever.invoke("What is Cacio e pepe?")

In [53]:
len(retriever_docs)
print(retriever_docs[5].page_content)

was ideal due to its extraordinary shelf life, black pepper was
used to stimulate heat receptors, and homemade noodles pro-
vided the carbohydrate intake.
Despite the short list of ingredients, preparing this dish
requires extra care. This is because cheese and water mix-
tures are notoriously prone to separation when heated, as seen
for example in Swiss cheese fondue [32]. In the context of
Cacio e pepe, this challenge has already been explored by
Dario Bressanini [33]. The most delicate step is the mixing
of starch-enriched water with grated cheese. One starts cook-
ing pasta noodles in boiling water, letting them release starch,
then extracts part of the water and starch solution. An essen-
tial procedure is to wait some time before mixing water and
cheese, to let the water cool down. This is because, at high
temperatures, cheese proteins can either form clumps upon
denaturation or simply aggregate [34, 35], therefore ruining


In [54]:
## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

In [117]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    temperature=0
)


In [118]:
system_prompt = ( 
    "You are assistant for question-answering tasks"
    "Use the following pieces of contect to answer the question at the end."
    "If you don't know the answer, just say that you don't know, don't try to manipulate "
    "Use three sentences maximum to answer"
    "\n\n"
    "{context}"
)


In [119]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Context:\n{context}\n\nQuestion:\n{input}"
)


In [120]:
## Soru-Cevap Zinziri Oluşturma (LLM + PROMPT)

In [121]:
from langchain_core.output_parsers import StrOutputParser

question_answering_chain = prompt | llm | StrOutputParser()


In [122]:
## RAG ZİNCİRİ OLUŞTURMA 

In [123]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever,
        "input": RunnablePassthrough()
    }
    | question_answering_chain
)


In [124]:
rag_chain.invoke("Explain the importance of temperature")


ChatGoogleGenerativeAIError: Error calling model 'models/gemini-1.5-flash' (NOT_FOUND): 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}